IMAGE SEGMENTATION = PARTITION IMAGES INTO DIFFERENT REGIONS

<!--  -->

Contours

Hierarchy in Contours:

    1. cv2.RETR_LIST - retrieves all contours (inner and outer)
    2. cv2.RETR_EXTERNAL - retrieves only external contours
    3. cv2.RETR_COMP - retrieves all in a 2 level heirarchy
    4. cv2.RETR_TREE - retrieves all in full hierarchy
    
Hierarchy is stored in the following format: [Next, Previous, Child, Parent]

In [68]:
import cv2 as cv, numpy as np

# Load images
shape_image = cv.imread("C://opencv/images/shapes.jpg")
donut_shape_image = cv.imread("C://opencv/images/shapes_donut.jpg")
bunchof_shapes = cv.imread("C://opencv/images/bunchofshapes.jpg")
house_image = cv.imread("c://opencv/images/house.jpg")
hand_image = cv.imread("c://opencv/images/hand.jpg")

In [102]:
cv.imshow("Original", shape_image)
cv.waitKey()

grey = cv.cvtColor(shape_image, cv.COLOR_BGR2GRAY)

# Find Canny edges
edge = cv.Canny(grey, 30, 200)
cv.imshow("Canny edges", edge)
cv.waitKey()

# get contours
# since findContours modifies the image, make a copy edge.copy()
edge_copy = edge.copy()

contours, heirarchy = cv.findContours(edge_copy, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
cv.imshow("Contours edges", edge_copy)
cv.waitKey()
# print(contours)
print("Number of contours: {}".format(len(contours)))

# draw contours. Use -1 as 3rd parameter to draw all
cv.drawContours(shape_image, contours, -1, (0,255,0), 3)
cv.imshow("Contours", shape_image)
cv.waitKey()

cv.destroyAllWindows()

Number of contours: 3


<!--  -->

SORTING CONTOURS

Useful in image processing

Sorting by area: assists in object recognition (using contour area):

    * Eliminates small contours that may be noise
    * Extract largest contours
 
Sorting by spacial position (Using contour centroid)

    * Sort charcters left to right
    * Process image in a specific order

In [3]:
cv.imshow("Original image", bunchof_shapes)
cv.waitKey()

# create a black image with the same dimension as our loaded image
blank_bunchof_shapes = np.zeros((bunchof_shapes.shape[0], bunchof_shapes.shape[1], 3))

# create a copy of original image
copy_bunchof_shapes = bunchof_shapes

#grayscale image
gray_bunchof_shapes = cv.cvtColor(bunchof_shapes, cv.COLOR_BGR2GRAY)

# Find canny edges
edgesof_gray = cv.Canny(gray_bunchof_shapes, 50 ,200)
cv.imshow("1 - Canny edges", edgesof_gray)
cv.waitKey(0)

# Find contours and print how many found
contours, hierarchy = cv.findContours(edgesof_gray.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
print("Number of contours found: {}".format(len(contours)))

# Draw all contours
cv.drawContours(bunchof_shapes, contours, -1, (0,255,0), 3)
cv.imshow("Contours on original", bunchof_shapes)
cv.waitKey()

# Contours over blank image
cv.drawContours(blank_bunchof_shapes, contours, -1, (0,255,0), 3)
cv.imshow("Contours on black", blank_bunchof_shapes)
cv.waitKey()

cv.destroyAllWindows()

Number of contours found: 4


<b>SORT by Area</b>

In [4]:
# function to display area
def get_contour_area(contours):
    all_areas = []
    for contour in contours:
        all_areas.append(cv.contourArea(contour))
    return all_areas

# Load image
image_of_shapes = cv.imread("C://opencv/images/bunchofshapes.jpg")
cv.imshow("Original shapes", image_of_shapes)
moded_image = image_of_shapes
cv.waitKey()

print("Areas of contours before sorting: {}".format(get_contour_area(contours)))

# Sort contours large to small
sorted_contours = sorted(contours, key=cv.contourArea, reverse=True)

print("Areas of contours after sorting: {}".format(get_contour_area(sorted_contours)))

# Iterate over contours and draw onw at a time
for c in sorted_contours:
    cv.drawContours(moded_image, [c], -1, (255,0,0), 3)
    cv.waitKey(0)
    cv.imshow("Contours by area", moded_image)

cv.waitKey(0)
cv.destroyAllWindows()

Areas of contours before sorting: [20587.5, 22901.5, 66579.5, 90222.0]
Areas of contours after sorting: [90222.0, 66579.5, 22901.5, 20587.5]


|
|
|

Sorting Contours From Left to Right

In [60]:


# Function to sort by position
def x_cord_contour(contours):
    if cv.contourArea(contours) > 0:
        M = cv.moments(contours)
        return (int(M["m10"]/M["m00"]))

def label_contour_center(image, c, i):
#   place a red circle in contour centre
    M = cv.moments(c)
    cx = int(M["m10"]/M["m00"])
    cy = int(M["m01"]/M["m00"])
    #draw contour on image head
    cv.circle(image, (cx,cy), 10, (0,0,255), -1)
    return image

original_image = image_of_shapes.copy()

# computer center of mass or centroid and draw them on our image
for i,c in enumerate(contours):
    orig = label_contour_center(image_of_shapes, c, i)

cv.imshow("Contour Centers", image_of_shapes)
cv.waitKey()

# Contours left-to-right sorting using x_cord_contour func
contour_left_right = sorted(contours, key=x_cord_contour, reverse=False)

# Label contours left_right
for i,c in enumerate(contour_left_right):
    cv.drawContours(original_image, [c], -1, (0,0,255), 3)
    M = cv.moments(c)
    cx = int(M["m10"]/M["m00"])
    cy = int(M["m01"]/M["m00"])
    cv.putText(original_image, str(i+1), (cx,cy), cv.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
    cv.imshow("Left to Right Contour", original_image)
    cv.waitKey(0)
    (x, y, h, w) = cv.boundingRect(c)
    
    #Crop contours and save images
    cropped_contour = original_image[y:y + h, x:x + w]
    image_name = "output_shape_number_" + str(i+1) + ".jpg"
    print(image_name)
    cv.imwrite(image_name, cropped_contour)

cv.destroyAllWindows()

ZeroDivisionError: float division by zero

<!--  -->


Approximating Contours and Convex Hall

cv2.approxPolyDP(contour, Approximation Accuracy, Closed)

    1. contour = contour we wish to approximate
    2. Approximation accuracy = accuracy of approx. Small value is better. usually < 5% of contour perimeter
    3. closed = boolean value to state open or closed approx. contour

In [31]:
orig_image = house_image.copy()
cv.imshow("Original Image", orig_image)
cv.waitKey(0)

# grayscale and binarize
gray = cv.cvtColor(house_image, cv.COLOR_BGR2GRAY)
ret, thresh = cv.threshold(gray, 127, 255, cv.THRESH_BINARY_INV)

# find contours
contours, hierarchy = cv.findContours(thresh.copy(), cv.RETR_LIST, cv.CHAIN_APPROX_NONE)

# iterate through each contour and compute bounding rectangle
for c in contours:
    x,y,w,h = cv.boundingRect(c)
    cv.rectangle(orig_image, (x,y), (x+w, y+h), (0,0,255), 0)
    cv.imshow("Bounding Rectangle", orig_image)

cv.waitKey(0)

# iterate through each contour and find approximate contour
for c in contours:
    #calculate accuracy as a % of the contour perimeter
    accuracy = .03 * cv.arcLength(c, True)
    approx = cv.approxPolyDP(c, accuracy, True)
    cv.drawContours(house_image, [approx], 0, (0,0,255), 2)
    cv.imshow("Approx Poly DP", house_image)

cv.waitKey(0)

cv.destroyAllWindows()

# 

<b>CONVEX HALL</b>

In [95]:
hand_image = cv.imread("c://opencv/images/hand.jpg")
gray = cv.cvtColor(hand_image, cv.COLOR_BGR2GRAY)
cv.imshow("Original Image", hand_image)
cv.waitKey(0)

# threashold the image
ret, thresh = cv.threshold(gray, 176, 255, 0)

# find contours
contours, heirarchy = cv.findContours(thresh.copy(), cv.RETR_LIST, cv.CHAIN_APPROX_NONE)

# sort contours by area nad remove the largestframe contour
n = len(contours) - 1 #or n = len(contours)
contours = sorted(contours, key=cv.contourArea, reverse=False)[:n]

# iterate through contours and draw the convex hall
for c in contours:
    hall = cv.convexHull(c)
    cv.drawContours(hand_image, [hall], 0, (20,255,25), 2)
    cv.imshow("Convex Hall", hand_image)

cv.waitKey(0)

cv.destroyAllWindows()

In [72]:
values = [2,4,5,1,13,87,34,9,0,3]

In [57]:
nval = sorted(values)
values[6]

34


<b>MATCHNING CONTOURS WITH SHAPES</b>

Match shapes with images even when distorted

<b>Shape Matching</b>

cv2.matchShapes(contour template, contour, method, method parameter)

    . contour template = ref contour we trying to find in new image
    . contour = individual contour we are checking against
    . method = type of contour matching (1,2,3)
    . method parameter = leave as 0.0 (not fully utilised in Python openCV)

In [126]:
template = cv.imread("c://opencv/images/4star.jpg", 0)
cv.imshow("Original Image", template)
cv.waitKey(0)

target = cv.imread("c://opencv/images/shapestomatch.jpg")
gray_target = cv.cvtColor(target, cv.COLOR_BGR2GRAY)

# threshold both images before finding contours
ret, thresh1 = cv.threshold(template, 127, 255, 0)
ret, thresh2 = cv.threshold(gray_target, 127, 255, 0)

# now find contours in template
contours, heirarchy = cv.findContours(thresh1, cv.RETR_CCOMP, cv.CHAIN_APPROX_SIMPLE)

# Now sort contours by area so as to remove largest(bounding windows frame)
sorted_contours = sorted(contours, key=cv.contourArea, reverse=True)

# extract second largest contour = template contour
template_contour = sorted_contours[1]

# extract contours from second target image
contours, heirarchy = cv.findContours(thresh2, cv.RETR_CCOMP, cv.CHAIN_APPROX_SIMPLE)

for c in contours:
    # iterate thru each contour in the target image
    # use cv.matchShapes to compare contour shapes
    match = cv.matchShapes(template_contour, c, 1, 0.0)
    print(match)
    if match < .15:
        closest_contour = c
        print(closest_contour.shape)
        
    else:
        closest_contour = []
    
cv.drawContours(target, [closest_contour], -1, (0, 255, 0), 3)
cv.imshow("Output", target)
cv.waitKey()

cv.destroyAllWindows()

0.16818605122199104
0.19946910256158912
0.18949760627309664
0.11101058276281539
(363, 1, 2)


In [111]:
cv.destroyAllWindows()

    |                |
    |                |
    |                |
    |                |
    V                V
    
 
 
Mini Project:

http://localhost:8888/notebooks/PythonClass/CompVis%20with%20OpenCV/4.%20Image%20Segmentation%20%26%20Contours/Mini%20Project%202%20-%20Identify%20shapes%20(Square%2C%20Rectangle%2C%20Circle%2C%20Traingle%2C%20Star).ipynb